In [12]:
from __future__ import division
import os
import cv2
import numpy as np
import sys
import pickle
from optparse import OptionParser
import time
from keras_frcnn import config
import keras_frcnn.resnet as nn
from keras import backend as K
from keras.layers import Input
from keras.models import Model
from keras_frcnn import roi_helpers
from keras_frcnn.simple_parser import get_data
from matplotlib import pyplot as plt
from scipy.misc import imread
from quiver_engine import server

sys.setrecursionlimit(40000)

config_output_filename = "config.pickle"

with open(config_output_filename, 'rb') as f_in:
	C = pickle.load(f_in)

# turn off any data augmentation at test time
C.use_horizontal_flips = False
C.use_vertical_flips = False
C.rot_90 = False

img_path = "apple_images/test_imgs"

def format_img_size(img, C):
	""" formats the image size based on config """
	img_min_side = float(C.im_size)
	(height,width,_) = img.shape
		
	if width <= height:
		ratio = img_min_side/width
		new_height = int(ratio * height)
		new_width = int(img_min_side)
	else:
		ratio = img_min_side/height
		new_width = int(ratio * width)
		new_height = int(img_min_side)
	img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_CUBIC)
	return img, ratio	

def format_img_channels(img, C):
	""" formats the image channels based on config """
	img = img[:, :, (2, 1, 0)]
	img = img.astype(np.float32)
	img[:, :, 0] -= C.img_channel_mean[0]
	img[:, :, 1] -= C.img_channel_mean[1]
	img[:, :, 2] -= C.img_channel_mean[2]
	img /= C.img_scaling_factor
	img = np.transpose(img, (2, 0, 1))
	img = np.expand_dims(img, axis=0)
	return img

def format_img(img, C):
	""" formats an image for model prediction based on config """
	img, ratio = format_img_size(img, C)
	img = format_img_channels(img, C)
	return img, ratio

# Method to transform the coordinates of the bounding box to its original size
def get_real_coordinates(ratio, x1, y1, x2, y2):

	real_x1 = int(round(x1 // ratio))
	real_y1 = int(round(y1 // ratio))
	real_x2 = int(round(x2 // ratio))
	real_y2 = int(round(y2 // ratio))

	return (real_x1, real_y1, real_x2 ,real_y2)

class_mapping = C.class_mapping

if 'bg' not in class_mapping:
	class_mapping['bg'] = len(class_mapping)

class_mapping = {v: k for k, v in class_mapping.items()}
print(class_mapping)
class_to_color = {class_mapping[v]: np.random.randint(0, 255, 3) for v in class_mapping}
C.num_rois = 32

if K.image_dim_ordering() == 'th':
	input_shape_img = (3, None, None)
	input_shape_features = (1024, None, None)
else:
	input_shape_img = (None, None, 3)
	input_shape_features = (None, None, 1024)


img_input = Input(shape=input_shape_img)
roi_input = Input(shape=(C.num_rois, 4))
feature_map_input = Input(shape=input_shape_features)

# define the base network (resnet here, can be VGG, Inception, etc)
shared_layers = nn.nn_base(img_input, trainable=True)

# define the RPN, built on the base layers
num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
rpn_layers = nn.rpn(shared_layers, num_anchors)

classifier = nn.classifier(feature_map_input, roi_input, C.num_rois, nb_classes=len(class_mapping), trainable=True)

model_rpn = Model(img_input, rpn_layers)
model_classifier_only = Model([feature_map_input, roi_input], classifier)

model_classifier = Model([feature_map_input, roi_input], classifier)

model_rpn.load_weights(C.model_path, by_name=True)
model_classifier.load_weights(C.model_path, by_name=True)
print(C.model_path)

model_rpn.compile(optimizer='sgd', loss='mse')
model_classifier.compile(optimizer='sgd', loss='mse')

{0: ' apple', 1: 'bg'}
mdel_frcnn2.hdf5


In [10]:
server.launch(model_rpn, temp_folder='./tmp_quiver/', input_folder='./apple_images/test_imgs/', port=8090)

Starting webserver from: /home/dl-user2/anaconda3/lib/python3.5/site-packages/quiver_engine
(224, 224, 3)


::ffff:109.205.109.75 - - [2017-07-06 15:28:38] "GET /inputs HTTP/1.1" 200 33991 0.005645
::ffff:109.205.109.75 - - [2017-07-06 15:28:38] "GET /model HTTP/1.1" 200 104213 0.144845
::ffff:109.205.109.75 - - [2017-07-06 15:28:40] "GET /input-file/IMG_20170625_175214818.jpg HTTP/1.1" 200 639507 0.325412
::ffff:109.205.109.75 - - [2017-07-06 15:28:40] "GET /input-file/20170625_181652.jpg HTTP/1.1" 200 740605 0.325155
::ffff:109.205.109.75 - - [2017-07-06 15:28:40] "GET /input-file/P_20170625_175435.jpg HTTP/1.1" 200 543844 0.327285
::ffff:109.205.109.75 - - [2017-07-06 15:28:40] "GET /input-file/P_20170625_181718.jpg HTTP/1.1" 200 604669 0.559972
::ffff:109.205.109.75 - - [2017-07-06 15:28:41] "GET /input-file/WP_20170625_18_14_19_Pro-0.jpg HTTP/1.1" 200 565605 0.764275
::ffff:109.205.109.75 - - [2017-07-06 15:28:41] "GET /input-file/IMG_20170625_175947-0.jpg HTTP/1.1" 200 609250 0.001721
::ffff:109.205.109.75 - - [2017-07-06 15:28:41] "GET /input-file/P_20170625_181547.jpg HTTP/1.1" 200 6

::ffff:109.205.109.75 - - [2017-07-06 15:28:50] "GET /input-file/P_20170625_183544.jpg HTTP/1.1" 200 574788 0.003742
::ffff:109.205.109.75 - - [2017-07-06 15:28:50] "GET /input-file/20170625_180712.jpg HTTP/1.1" 200 741697 0.221375
::ffff:109.205.109.75 - - [2017-07-06 15:28:50] "GET /input-file/IMG_20170625_180513-0.jpg HTTP/1.1" 200 449224 0.002759
::ffff:109.205.109.75 - - [2017-07-06 15:28:50] "GET /input-file/WP_20170625_18_04_40_Pro-0.jpg HTTP/1.1" 200 563159 0.008845
::ffff:109.205.109.75 - - [2017-07-06 15:28:50] "GET /input-file/IMG_20170625_180047-0.jpg HTTP/1.1" 200 592602 0.010878
::ffff:109.205.109.75 - - [2017-07-06 15:28:50] "GET /input-file/P_20170625_184426.jpg HTTP/1.1" 200 581484 0.005112
::ffff:109.205.109.75 - - [2017-07-06 15:28:51] "GET /input-file/WP_20170625_18_21_09_Pro-0.jpg HTTP/1.1" 200 494980 0.004196
::ffff:109.205.109.75 - - [2017-07-06 15:28:51] "GET /input-file/P_20170625_180614.jpg HTTP/1.1" 200 592302 0.005371
::ffff:109.205.109.75 - - [2017-07-06 15

::ffff:109.205.109.75 - - [2017-07-06 15:28:59] "GET /input-file/20170625_181040.jpg HTTP/1.1" 200 734138 0.001859
::ffff:109.205.109.75 - - [2017-07-06 15:28:59] "GET /input-file/WP_20170625_18_24_38_Pro-0.jpg HTTP/1.1" 200 528083 0.003846
::ffff:109.205.109.75 - - [2017-07-06 15:28:59] "GET /input-file/IMG_20170625_175907-0.jpg HTTP/1.1" 200 599233 0.002588
::ffff:109.205.109.75 - - [2017-07-06 15:28:59] "GET /input-file/WP_20170625_18_22_36_Pro-0.jpg HTTP/1.1" 200 587164 0.003142
KeyboardInterrupt
Thu Jul  6 15:28:59 2017


KeyboardInterrupt: 

In [13]:
server.launch(model_rpn, temp_folder='./tmp_quiver', input_folder='./test_quiver', port=8090)

Starting webserver from: /home/dl-user2/anaconda3/lib/python3.5/site-packages/quiver_engine
(224, 224, 3)


::ffff:109.205.109.75 - - [2017-07-06 15:29:44] "GET /inputs HTTP/1.1" 200 33991 0.002720
::ffff:109.205.109.75 - - [2017-07-06 15:29:44] "GET /model HTTP/1.1" 200 104213 0.022682
::ffff:109.205.109.75 - - [2017-07-06 15:29:47] "GET /input-file/20170625_180825.jpg HTTP/1.1" 404 374 0.000843
::ffff:109.205.109.75 - - [2017-07-06 15:29:47] "GET /input-file/IMG_20170625_182359-0.jpg HTTP/1.1" 404 374 0.000561
::ffff:109.205.109.75 - - [2017-07-06 15:29:47] "GET /input-file/20170625_182619.jpg HTTP/1.1" 404 374 0.000484
::ffff:109.205.109.75 - - [2017-07-06 15:29:47] "GET /input-file/IMG_20170625_175528726.jpg HTTP/1.1" 404 374 0.000484
::ffff:109.205.109.75 - - [2017-07-06 15:29:47] "GET /input-file/IMG_20170625_180950-0.jpg HTTP/1.1" 404 374 0.000760
::ffff:109.205.109.75 - - [2017-07-06 15:29:47] "GET /input-file/20170625_181720.jpg HTTP/1.1" 404 374 0.000769
::ffff:109.205.109.75 - - [2017-07-06 15:29:47] "GET /input-file/20170625_180110.jpg HTTP/1.1" 404 374 0.001043
::ffff:109.205.10

::ffff:109.205.109.75 - - [2017-07-06 15:29:50] "GET /input-file/P_20170625_180824.jpg HTTP/1.1" 404 374 0.002549
::ffff:109.205.109.75 - - [2017-07-06 15:29:50] "GET /input-file/WP_20170625_18_46_24_Pro-0.jpg HTTP/1.1" 404 374 0.001304
::ffff:109.205.109.75 - - [2017-07-06 15:29:50] "GET /input-file/WP_20170625_18_25_13_Pro-0.jpg HTTP/1.1" 404 374 0.001220
::ffff:109.205.109.75 - - [2017-07-06 15:29:50] "GET /input-file/20170625_181832.jpg HTTP/1.1" 404 374 0.001455
::ffff:109.205.109.75 - - [2017-07-06 15:29:50] "GET /input-file/WP_20170625_18_05_11_Pro-0.jpg HTTP/1.1" 404 374 0.000881
::ffff:109.205.109.75 - - [2017-07-06 15:29:50] "GET /input-file/20170625_181701-0.jpg HTTP/1.1" 404 374 0.001562
::ffff:109.205.109.75 - - [2017-07-06 15:29:51] "GET /input-file/WP_20170625_18_20_43_Pro-0.jpg HTTP/1.1" 404 374 0.003005
::ffff:109.205.109.75 - - [2017-07-06 15:29:51] "GET /input-file/IMG_20170625_175918-0.jpg HTTP/1.1" 404 374 0.003834
::ffff:109.205.109.75 - - [2017-07-06 15:29:51] "G

::ffff:109.205.109.75 - - [2017-07-06 15:29:54] "GET /input-file/20170625_181527-0.jpg HTTP/1.1" 404 374 0.001814
::ffff:109.205.109.75 - - [2017-07-06 15:29:54] "GET /input-file/P_20170625_182239.jpg HTTP/1.1" 404 374 0.001677
::ffff:109.205.109.75 - - [2017-07-06 15:29:54] "GET /input-file/P_20170625_175431.jpg HTTP/1.1" 404 374 0.001299
::ffff:109.205.109.75 - - [2017-07-06 15:29:54] "GET /input-file/20170625_180354.jpg HTTP/1.1" 404 374 0.001409
::ffff:109.205.109.75 - - [2017-07-06 15:29:54] "GET /input-file/IMG_20170625_181044-0.jpg HTTP/1.1" 404 374 0.000603
::ffff:109.205.109.75 - - [2017-07-06 15:29:54] "GET /input-file/20170625_181010-0.jpg HTTP/1.1" 404 374 0.001590
::ffff:109.205.109.75 - - [2017-07-06 15:29:54] "GET /input-file/IMG_20170625_175943-0.jpg HTTP/1.1" 404 374 0.001638
::ffff:109.205.109.75 - - [2017-07-06 15:29:54] "GET /input-file/P_20170625_181920.jpg HTTP/1.1" 404 374 0.000770
::ffff:109.205.109.75 - - [2017-07-06 15:29:54] "GET /input-file/P_20170625_181723

::ffff:109.205.109.75 - - [2017-07-06 15:29:57] "GET /input-file/IMG_20170625_180201-0.jpg HTTP/1.1" 404 374 0.001366
::ffff:109.205.109.75 - - [2017-07-06 15:29:57] "GET /input-file/WP_20170625_18_50_01_Pro-0.jpg HTTP/1.1" 404 374 0.001072
::ffff:109.205.109.75 - - [2017-07-06 15:29:57] "GET /input-file/WP_20170625_18_43_49_Pro-0.jpg HTTP/1.1" 404 374 0.007144
::ffff:109.205.109.75 - - [2017-07-06 15:29:57] "GET /input-file/IMG_20170625_175921-0.jpg HTTP/1.1" 404 374 0.001515
::ffff:109.205.109.75 - - [2017-07-06 15:29:57] "GET /input-file/20170625_182458.jpg HTTP/1.1" 404 374 0.000828
::ffff:109.205.109.75 - - [2017-07-06 15:29:57] "GET /input-file/20170625_180823-0.jpg HTTP/1.1" 404 374 0.001528
::ffff:109.205.109.75 - - [2017-07-06 15:29:57] "GET /input-file/P_20170625_181714.jpg HTTP/1.1" 404 374 0.000500
::ffff:109.205.109.75 - - [2017-07-06 15:29:57] "GET /input-file/P_20170625_181303.jpg HTTP/1.1" 404 374 0.000484
::ffff:109.205.109.75 - - [2017-07-06 15:29:57] "GET /input-file

...

(224, 224, 3)


::ffff:109.205.109.75 - - [2017-07-06 15:30:55] "GET /layer/zero_padding2d_1/IMG_20170625_175430-0.jpg HTTP/1.1" 200 308 0.520721
::ffff:109.205.109.75 - - [2017-07-06 15:30:55] "GET /temp-file/conv1_45_IMG_20170625_175430-0.jpg.png HTTP/1.1" 200 10359 0.001358
::ffff:109.205.109.75 - - [2017-07-06 15:30:55] "GET /temp-file/conv1_49_IMG_20170625_175430-0.jpg.png HTTP/1.1" 200 11226 0.001116
::ffff:109.205.109.75 - - [2017-07-06 15:30:55] "GET /temp-file/conv1_55_IMG_20170625_175430-0.jpg.png HTTP/1.1" 200 10393 0.000972
::ffff:109.205.109.75 - - [2017-07-06 15:30:55] "GET /temp-file/conv1_57_IMG_20170625_175430-0.jpg.png HTTP/1.1" 200 11428 0.001411
::ffff:109.205.109.75 - - [2017-07-06 15:30:55] "GET /temp-file/conv1_42_IMG_20170625_175430-0.jpg.png HTTP/1.1" 200 10382 0.001663
::ffff:109.205.109.75 - - [2017-07-06 15:30:55] "GET /temp-file/conv1_47_IMG_20170625_175430-0.jpg.png HTTP/1.1" 200 11423 0.001147
::ffff:109.205.109.75 - - [2017-07-06 15:30:55] "GET /temp-file/conv1_59_IMG_2

(224, 224, 3)


::ffff:109.205.109.75 - - [2017-07-06 15:30:57] "GET /layer/activation_1/IMG_20170625_175430-0.jpg HTTP/1.1" 200 3462 0.744220
KeyboardInterrupt
Thu Jul  6 15:43:54 2017


KeyboardInterrupt: 

In [ ]:
all_imgs = []

classes = {}

bbox_threshold = 0.8

visualise = True

for idx, img_name in enumerate(sorted(os.listdir(img_path))):
	if not img_name.lower().endswith(('.bmp', '.jpeg', '.jpg', '.png', '.tif', '.tiff')):
		continue
	print(img_name)
	st = time.time()
	filepath = os.path.join(img_path,img_name)

# 	img = cv2.imread(filepath)
	img = imread(filepath)

	X, ratio = format_img(img, C)

	if K.image_dim_ordering() == 'tf':
		X = np.transpose(X, (0, 2, 3, 1))

	# get the feature maps and output from the RPN
	[Y1, Y2, F] = model_rpn.predict(X)
	

	R = roi_helpers.rpn_to_roi(Y1, Y2, C, K.image_dim_ordering(), overlap_thresh=0.7)

	# convert from (x1,y1,x2,y2) to (x,y,w,h)
	R[:, 2] -= R[:, 0]
	R[:, 3] -= R[:, 1]

	# apply the spatial pyramid pooling to the proposed regions
	bboxes = {}
	probs = {}

	for jk in range(R.shape[0]//C.num_rois + 1):
		ROIs = np.expand_dims(R[C.num_rois*jk:C.num_rois*(jk+1), :], axis=0)
		if ROIs.shape[1] == 0:
			break

		if jk == R.shape[0]//C.num_rois:
			#pad R
			curr_shape = ROIs.shape
			target_shape = (curr_shape[0],C.num_rois,curr_shape[2])
			ROIs_padded = np.zeros(target_shape).astype(ROIs.dtype)
			ROIs_padded[:, :curr_shape[1], :] = ROIs
			ROIs_padded[0, curr_shape[1]:, :] = ROIs[0, 0, :]
			ROIs = ROIs_padded

		[P_cls, P_regr] = model_classifier_only.predict([F, ROIs])

		for ii in range(P_cls.shape[1]):

			if np.max(P_cls[0, ii, :]) < bbox_threshold or np.argmax(P_cls[0, ii, :]) == (P_cls.shape[2] - 1):
				continue

			cls_name = class_mapping[np.argmax(P_cls[0, ii, :])]

			if cls_name not in bboxes:
				bboxes[cls_name] = []
				probs[cls_name] = []

			(x, y, w, h) = ROIs[0, ii, :]

			cls_num = np.argmax(P_cls[0, ii, :])
			try:
				(tx, ty, tw, th) = P_regr[0, ii, 4*cls_num:4*(cls_num+1)]
				tx /= C.classifier_regr_std[0]
				ty /= C.classifier_regr_std[1]
				tw /= C.classifier_regr_std[2]
				th /= C.classifier_regr_std[3]
				x, y, w, h = roi_helpers.apply_regr(x, y, w, h, tx, ty, tw, th)
			except:
				pass
			bboxes[cls_name].append([C.rpn_stride*x, C.rpn_stride*y, C.rpn_stride*(x+w), C.rpn_stride*(y+h)])
			probs[cls_name].append(np.max(P_cls[0, ii, :]))

	all_dets = []

	for key in bboxes:
		bbox = np.array(bboxes[key])

		new_boxes, new_probs = roi_helpers.non_max_suppression_fast(bbox, np.array(probs[key]), overlap_thresh=0.5)
		for jk in range(new_boxes.shape[0]):
			(x1, y1, x2, y2) = new_boxes[jk,:]

			(real_x1, real_y1, real_x2, real_y2) = get_real_coordinates(ratio, x1, y1, x2, y2)

			cv2.rectangle(img,(real_x1, real_y1), (real_x2, real_y2), (int(class_to_color[key][0]), int(class_to_color[key][1]), int(class_to_color[key][2])),2)

			textLabel = '{}: {}'.format(key,int(100*new_probs[jk]))
			all_dets.append((key,100*new_probs[jk]))

			(retval,baseLine) = cv2.getTextSize(textLabel,cv2.FONT_HERSHEY_PLAIN,1,1)
			textOrg = (real_x1, real_y1-0)

# 			cv2.rectangle(img, (textOrg[0] - 5, textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (0, 0, 0), 2)
# 			cv2.rectangle(img, (textOrg[0] - 5,textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (255, 255, 255), -1)
# 			cv2.putText(img, textLabel, textOrg, cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 1)

	print('Elapsed time = {}'.format(time.time() - st))
	print(all_dets)
# 	cv2.imshow('img', img)
#	cv2.imwrite('./results_imgs/{}.png'.format(idx),img)
	print(type(img))
	plt.figure(figsize=(20, 8))
# 	plt.subplot(1,2,1)
	plt.imshow(img)
# 	plt.subplot(1,2,2)
# 	plt.imshow(imread(filepath))
	plt.show()